# 1. 코로나 kaggle (20-01-20 ~ 21-04-29)
## 101일*17 = 1717행
https://www.kaggle.com/kimjihoo/coronavirusdataset

In [1]:
import pandas as pd

In [2]:
df2 = pd.read_csv("TimeProvince.csv")
df2

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0
...,...,...,...,...,...,...
2766,2020-06-30,0,Jeollabuk-do,27,21,0
2767,2020-06-30,0,Jeollanam-do,24,19,0
2768,2020-06-30,0,Gyeongsangbuk-do,1389,1328,54
2769,2020-06-30,0,Gyeongsangnam-do,134,128,0


In [3]:
df2.drop(["time", "released", "deceased"], axis = 1, inplace=True)

In [4]:
df2 = df2.drop(df2[df2['date'] >= '2020-04-30'].index)
df2

,date,province,confirmed
0,2020-01-20,Seoul,0
1,2020-01-20,Busan,0
2,2020-01-20,Daegu,0
3,2020-01-20,Incheon,1
4,2020-01-20,Gwangju,0
...,...,...,...
1712,2020-04-29,Jeollabuk-do,18
1713,2020-04-29,Jeollanam-do,15
1714,2020-04-29,Gyeongsangbuk-do,1365
1715,2020-04-29,Gyeongsangnam-do,117


In [5]:
확진 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(len(df2) - 17):
    확진.append(int(df2.confirmed[i+17]) - int(df2.confirmed[i]))
len(확진)

1717

In [6]:
len(df2)

1717

In [7]:
df2["new_confirmed"] = 확진

In [8]:
df2.drop("confirmed", axis = 1, inplace=True)
df2

,date,province,new_confirmed
0,2020-01-20,Seoul,0
1,2020-01-20,Busan,0
2,2020-01-20,Daegu,0
3,2020-01-20,Incheon,0
4,2020-01-20,Gwangju,0
...,...,...,...
1712,2020-04-29,Jeollabuk-do,0
1713,2020-04-29,Jeollanam-do,0
1714,2020-04-29,Gyeongsangbuk-do,0
1715,2020-04-29,Gyeongsangnam-do,0


In [9]:
df2.to_csv("corona.csv", mode='w', index=False)

# 2. 코로나 api (20-04-30 ~ 21-03-31)
## 336일*17 = 5712행
https://www.data.go.kr/data/15043378/openapi.do

In [10]:
# url을 가져와서 문서로 만드는 작업
import requests
from bs4 import BeautifulSoup as bs

In [11]:
#아래키는 본인키로 추후 변경하세요..

key='QN0hRwa2l5a7/tz6zGDUQuiQ4AryFMJ2zghxr4MpH9nyDWxF1aL362ZxYVqUIPuAOT2Cg5pbTYspNmJfcHLcHg=='
numRows=1000
staD=str(20200430)
endD=str(20210331)


url1='http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?serviceKey='+ key
url2='&pageNo=1&numOfRows='+str(numRows)
url3='&startCreateDt='+staD+'&endCreateDt='+endD
url=url1+url2+url3   

print(url)

http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?serviceKey=QN0hRwa2l5a7/tz6zGDUQuiQ4AryFMJ2zghxr4MpH9nyDWxF1aL362ZxYVqUIPuAOT2Cg5pbTYspNmJfcHLcHg==&pageNo=1&numOfRows=1000&startCreateDt=20200430&endCreateDt=20210331


In [12]:
xml=requests.get(url)
soup=bs(xml.text,'html.parser')
#soup

In [13]:
data=soup.find_all('item')
con_lst=[]

for item in data:
    DEF_CNT = item.find('defcnt').text # 확진자수'
    GUBUN = item.find('gubunen').text # 지역'
    INC_DEC = item.find('incdec').text #전일대비 증감'
    STD_DAY = item.find('createdt').text[:10] # 기준일'

    con_lst.append({'date':STD_DAY,'province':GUBUN,'new_confirmed':INC_DEC})

In [14]:
# 아래와 같이 해도 됨. 그러면 con_lst 순서대로 나오고 columns를 하면 내가 순서를 변경할 수 있음.
df = pd.DataFrame(con_lst)

df = pd.DataFrame(con_lst, columns = ['date', 'province','new_confirmed'])


df.fillna(0)

,date,province,new_confirmed
0,2021-03-31,Lazaretto,7
1,2021-03-31,Jeju,2
2,2021-03-31,Gyeongsangnam-do,22
3,2021-03-31,Gyeongsangbuk-do,6
4,2021-03-31,Jeollanam-do,0
...,...,...,...
6378,2020-04-30,Incheon,0
6379,2020-04-30,Daegu,0
6380,2020-04-30,Busan,0
6381,2020-04-30,Seoul,0


In [15]:
indexNames = df[ (df['province'] == 'Lazaretto')
                | (df['province'] == 'Total') ].index
df.drop(indexNames , inplace=True)

In [16]:
df

,date,province,new_confirmed
1,2021-03-31,Jeju,2
2,2021-03-31,Gyeongsangnam-do,22
3,2021-03-31,Gyeongsangbuk-do,6
4,2021-03-31,Jeollanam-do,0
5,2021-03-31,Jeollabuk-do,20
...,...,...,...
6377,2020-04-30,Gwangju,0
6378,2020-04-30,Incheon,0
6379,2020-04-30,Daegu,0
6380,2020-04-30,Busan,0


In [17]:
df.sort_values(by = 'date', inplace = True, ignore_index = True)
df

,date,province,new_confirmed
0,2020-04-30,Seoul,0
1,2020-04-30,Jeju,0
2,2020-04-30,Gyeongsangnam-do,0
3,2020-04-30,Gyeongsangbuk-do,0
4,2020-04-30,Jeollanam-do,0
...,...,...,...
5707,2021-03-31,Daegu,15
5708,2021-03-31,Busan,58
5709,2021-03-31,Seoul,154
5710,2021-03-31,Gyeonggi-do,111


In [18]:
df.to_csv("corona.csv", mode='a', index=False, header = False)

# 3. 총 기간 코로나 csv
## 437일 * 17지역 = 7429행

In [19]:
df_all = pd.read_csv("corona.csv")
df_all

,date,province,new_confirmed
0,2020-01-20,Seoul,0
1,2020-01-20,Busan,0
2,2020-01-20,Daegu,0
3,2020-01-20,Incheon,0
4,2020-01-20,Gwangju,0
...,...,...,...
7424,2021-03-31,Daegu,15
7425,2021-03-31,Busan,58
7426,2021-03-31,Seoul,154
7427,2021-03-31,Gyeonggi-do,111
